In [1]:
pip install mlflow transformers torch datasets flask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.

In [ ]:
!mlflow server --host 0.0.0.0 --port 5000

[2025-02-26 11:56:08 +0000] [5731] [INFO] Starting gunicorn 23.0.0
[2025-02-26 11:56:08 +0000] [5731] [INFO] Listening at: http://0.0.0.0:5000 (5731)
[2025-02-26 11:56:08 +0000] [5731] [INFO] Using worker: sync
[2025-02-26 11:56:08 +0000] [5732] [INFO] Booting worker with pid: 5732
[2025-02-26 11:56:08 +0000] [5733] [INFO] Booting worker with pid: 5733
[2025-02-26 11:56:08 +0000] [5734] [INFO] Booting worker with pid: 5734
[2025-02-26 11:56:08 +0000] [5735] [INFO] Booting worker with pid: 5735


In [ ]:
import mlflow
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Set MLflow tracking URI (local or remote server)
# Ensure MLflow server is running: `mlflow server --host 0.0.0.0 --port 5000`
mlflow.set_tracking_uri("http://127.0.0.1:5000") # Verify URI if server is running elsewhere

# Load model and dataset
model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = load_dataset("ag_news")

# Tokenization function
def tokenize(batch):
  return tokenizer(batch['text'], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize, batched=True)

# Load Model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2.

)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].shuffle(seed=42).select(range(1000)),
    eval_dataset=tokenized_dataset["test"].shuffle(seed=42).select(range(100))
)

# Train and log to MLflow
with mlflow.start_run():
  trainer.train()

  # Log model
  mlflow.transformers.log_model(
      transformers_model=model,
      artificat_path="model",
      task="text-classification",
      tokenizer=tokenizer
  )

  # log metrics
  metrics = trainer.evaluate() # Changed from trainer.evaluates() to trainer.evaluate()
  for key, value in metrics.items():
    mlflow.log_metric(key, value)

  # Register the model
  mlflow.register_model("runs:/{}/model".format(mlflow.active_run().info.run_id), "LLM-Text-Classifier")

In [ ]:
# Serve the Model via MLflow Deployments (REST API)
mlflow models serve -m "models:/LLM-Text-Classifier/Production" -p 5001

In [ ]:
# Implement Model Governance and Access Control
from flask import Flask, request, jsonify
import mlflow.pyfunc

app = flask(__name__)

# Load the MLflow model
model = mlflow.pyfunc.load_model("models:/LLM-Text-Classifier/Production")

# API Key for authentication
API_KEY = "secure-api-key"

@app.route('/predict', methods=['POST'])
def predict():
  if request.headers.get("x-api-key") != API_KEY:
    return jsonify({"error": "Unauthorized"}), 401

  data = request.json
  prediction = model.predict([data['text']])

  return jsonify({"prediction": prediciton.tolist()})

if __name__ == '__main__':
  app,run(host="0.0.0.0", port=5002)